In [ ]:
import requests
from bs4 import BeautifulSoup

url = "https://mixkit.co/free-sound-effects/"
headers = {"User-Agent": "Mozilla/5.0"}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

nav_div = soup.find("div", {"data-test-id": "global-navigation--sfx"})

categories_data = []

# Each category block
category_wrappers = nav_div.find_all("div", class_="global-navigation__link-wrapper")

for wrapper in category_wrappers:
    # Get main category
    main_link = wrapper.find("a", class_="global-navigation__link")
    if not main_link:
        continue
    category_name = main_link.text.strip()
    category_href = "https://mixkit.co" + main_link["href"]

    # Get subcategories
    sub_links = wrapper.find_all("a", class_="global-navigation__link")[1:]
    subcategories = [
        {
            "name": sub.text.strip(),
            "url": "https://mixkit.co" + sub["href"]
        }
        for sub in sub_links
    ]

    categories_data.append({
        "category": category_name,
        "url": category_href,
        "subcategories": subcategories
    })

# Print output
for cat in categories_data:
    # print(f"\n🟦 {cat['category']} ({cat['url']})")
    # for sub in cat["subcategories"]:
    #     # print(f"   └── {sub['name']} ({sub['url']})")
    #     print({sub}, step="")
    print(",".join(str(i['name']) for i in cat["subcategories"]))

In [28]:
subcategories = [
    "Blow", "Cinematic", "Glitch", "Impact", "Slide", "Spin", "Stomp", "Sweep", "Swish", "Swoosh", "Thud", "Whip", "Whoosh", "Woosh", "Zoom",
    "Ambience", "Beach", "Desert", "Fire", "Forest", "Jungle", "Morning", "Mountain", "Night", "Rain", "Rivers", "Safari", "Scary Woods", "Sea", "Storm", "Swell", "Thunder", "Volcano", "Water", "Waterfall", "Waves", "Wind", "Wood", "Splash", "Ocean", "Lightning",
    "Bleep", "Buzzer", "Camera", "Click", "Clock", "Keyboard", "Drone", "Electricity", "Error", "High Tech", "Interface", "Laptop", "Light", "Movie Projector", "Radio", "Rewind", "Robot", "Static", "Tape Machine", "Tones", "TV", "Type", "Typewriter", "Ding",
    "Fart", "Laugh", "Cartoon", "Happy",
    "Alarm", "Alerts", "Beep", "Phone", "Phone Ring",
    "Bird", "Cat", "Chicken", "Cow", "Crickets", "Dog", "Farm", "Horse", "Insect", "Lion", "Monkey", "Mouse", "Roar", "Wild", "Wolf", "Pig",
    "Airport", "Bike", "Boat", "Bus", "Car", "Engine", "Flying", "Helicopter", "Motorcycle", "Race Car", "Ship", "Siren", "Skateboard", "Tanks", "Time Machine", "Tractor", "Traffic", "Train", "Tram", "Car Horn", "Truck",
    "Applause", "Baby", "Breathe", "Children", "Clapping", "Cough", "Crowd", "Cry", "Eat", "Falling", "Footsteps", "Gasp", "Heartbeat", "Hum", "Hurt", "Kiss", "Moan", "Party", "Scream", "Public Places", "Voices", "Yawn", "Teeth", "Thank You", "Sad", "Sneeze", "Terror", "Jump", "Cheer", "Walk", "Run",
    "Battle", "Battle Ambience", "Bomb", "Explosion", "Fight", "Gun", "Hit", "Medieval Battle", "Rocket", "Sword", "War", "Boom", "Crash",
    "Bass", "Bell", "Brass", "Chimes", "Choir", "Church Bell", "Cymbal", "DJ Record Scratch", "Drum", "Flute", "Guitar", "Horn", "Music", "Orchestra", "Rock", "Sleigh Bells", "Synth", "Violins", "Whistle", "Suspense Music", "Tap", "Piano",
    "Appliances", "Bar", "Bathroom", "Bedroom", "Church", "City", "Cleaning", "Cooking", "Creaking Floorboard", "Doorbell", "Doors", "Drink", "Emergency", "Food", "Garden", "Hall", "Hospital", "Kitchen", "Office", "Restaurant", "School", "Supermarket", "Sweeping", "Girl",
    "Arcade", "Arcade Sport", "Asteroid", "Athletics", "Ball", "Basketball", "Boxing", "Casino", "Game Over", "Game Show", "Goal", "Golf", "Light Saber", "Lose", "Monster", "Player", "Punch", "Slot Machine", "Soccer", "Space Shooter", "Sports", "Toy", "Video Game", "Win", "Correct", "Laser", "Wrong", "Shot",
    "Scary", "Balloon", "Break", "Bubbles", "Coin", "Construction", "Countdown", "Cute", "Fairy", "Festive", "Fireworks", "Intro", "Key", "Halloween", "Lock", "Magic", "Metal", "Movie", "Page", "Paper", "Police", "Pop", "Sci-Fi", "Sparkle", "Spell", "Squeak", "Tools", "White Noise", "Write", "Horror", "Money", "Glass", "Wow", "Drop"
]
print(len(subcategories))

263


In [ ]:
import requests
from bs4 import BeautifulSoup
import os
import json
from tqdm import tqdm

base_url = "https://mixkit.co"
subcategories = [
    "Blow", "Cinematic", "Glitch", "Impact", "Slide", "Spin", "Stomp", "Sweep", "Swish", "Swoosh", "Thud", "Whip", "Whoosh", "Woosh", "Zoom",
    "Ambience", "Beach", "Desert", "Fire", "Forest", "Jungle", "Morning", "Mountain", "Night", "Rain", "Rivers", "Safari", "Scary Woods", "Sea", "Storm", "Swell", "Thunder", "Volcano", "Water", "Waterfall", "Waves", "Wind", "Wood", "Splash", "Ocean", "Lightning",
    "Bleep", "Buzzer", "Camera", "Click", "Clock", "Keyboard", "Drone", "Electricity", "Error", "High Tech", "Interface", "Laptop", "Light", "Movie Projector", "Radio", "Rewind", "Robot", "Static", "Tape Machine", "Tones", "TV", "Type", "Typewriter", "Ding",
    "Fart", "Laugh", "Cartoon", "Happy",
    "Alarm", "Alerts", "Beep", "Phone", "Phone Ring",
    "Bird", "Cat", "Chicken", "Cow", "Crickets", "Dog", "Farm", "Horse", "Insect", "Lion", "Monkey", "Mouse", "Roar", "Wild", "Wolf", "Pig",
    "Airport", "Bike", "Boat", "Bus", "Car", "Engine", "Flying", "Helicopter", "Motorcycle", "Race Car", "Ship", "Siren", "Skateboard", "Tanks", "Time Machine", "Tractor", "Traffic", "Train", "Tram", "Car Horn", "Truck",
    "Applause", "Baby", "Breathe", "Children", "Clapping", "Cough", "Crowd", "Cry", "Eat", "Falling", "Footsteps", "Gasp", "Heartbeat", "Hum", "Hurt", "Kiss", "Moan", "Party", "Scream", "Public Places", "Voices", "Yawn", "Teeth", "Thank You", "Sad", "Sneeze", "Terror", "Jump", "Cheer", "Walk", "Run",
    "Battle", "Battle Ambience", "Bomb", "Explosion", "Fight", "Gun", "Hit", "Medieval Battle", "Rocket", "Sword", "War", "Boom", "Crash",
    "Bass", "Bell", "Brass", "Chimes", "Choir", "Church Bell", "Cymbal", "DJ Record Scratch", "Drum", "Flute", "Guitar", "Horn", "Music", "Orchestra", "Rock", "Sleigh Bells", "Synth", "Violins", "Whistle", "Suspense Music", "Tap", "Piano",
    "Appliances", "Bar", "Bathroom", "Bedroom", "Church", "City", "Cleaning", "Cooking", "Creaking Floorboard", "Doorbell", "Doors", "Drink", "Emergency", "Food", "Garden", "Hall", "Hospital", "Kitchen", "Office", "Restaurant", "School", "Supermarket", "Sweeping", "Girl",
    "Arcade", "Arcade Sport", "Asteroid", "Athletics", "Ball", "Basketball", "Boxing", "Casino", "Game Over", "Game Show", "Goal", "Golf", "Light Saber", "Lose", "Monster", "Player", "Punch", "Slot Machine", "Soccer", "Space Shooter", "Sports", "Toy", "Video Game", "Win", "Correct", "Laser", "Wrong", "Shot",
    "Scary", "Balloon", "Break", "Bubbles", "Coin", "Construction", "Countdown", "Cute", "Fairy", "Festive", "Fireworks", "Intro", "Key", "Halloween", "Lock", "Magic", "Metal", "Movie", "Page", "Paper", "Police", "Pop", "Sci-Fi", "Sparkle", "Spell", "Squeak", "Tools", "White Noise", "Write", "Horror", "Money", "Glass", "Wow", "Drop"
]
output_dir = "mixkit_sounds"  # write each dir name root name 
os.makedirs(output_dir, exist_ok=True)

sound_data = []
sound_id = 375  # to increment for each sound

for sub in subcategories:
    sub_url = f"{base_url}/free-sound-effects/{sub}/"
    folder = os.path.join(output_dir, sub)
    os.makedirs(folder, exist_ok=True)

    response = requests.get(sub_url)
    soup = BeautifulSoup(response.text, "html.parser")
    sound_cards = soup.select('div.item-grid__item')

    for card in tqdm(sound_cards, desc=f"Downloading from {sub}"):
        # Extract title
        title_tag = card.select_one("h2.item-grid-card__title")
        if not title_tag:
            continue
        title = title_tag.text.strip().replace(" ", "_").replace("/", "-")

        # Extract mp3 URL
        audio_div = card.select_one('[data-audio-player-preview-url-value]')
        if not audio_div:
            continue
        mp3_url = audio_div['data-audio-player-preview-url-value']

        # Extract duration
        duration_tag = card.select_one('[data-test-id="duration"]')
        duration_text = duration_tag.text.strip() if duration_tag else "0:00"
        mins, secs = map(int, duration_text.split(":"))
        duration_seconds = mins * 60 + secs

        # Define file path
        filename = f"{title}.mp3"
        file_path = os.path.join(folder, filename)

        # Download file
        try:
            r = requests.get(mp3_url)
            with open(file_path, "wb") as f:
                f.write(r.content)
        except Exception as e:
            print(f"Failed to download {mp3_url}: {e}")
            continue

        # Store metadata
        sound_data.append({
            "id": sound_id,
            "name": title,
            "duration": duration_seconds,
            "file_path": os.path.abspath(file_path)
        })
        sound_id += 1

# Save metadata to JSON
with open(os.path.join(output_dir, "sounds_metadata.json"), "a") as f:
    json.dump(sound_data, f, indent=2)

print("Download and metadata collection complete.")
print(f"sound id : {sound_id}")

In [29]:
# fetch url 


import requests
from bs4 import BeautifulSoup
import os
import json
from tqdm import tqdm

base_url = "https://mixkit.co"
subcategories = [
    "Blow", "Cinematic", "Glitch", "Impact", "Slide", "Spin", "Stomp", "Sweep", "Swish", "Swoosh", "Thud", "Whip", "Whoosh", "Woosh", "Zoom",
    "Ambience", "Beach", "Desert", "Fire", "Forest", "Jungle", "Morning", "Mountain", "Night", "Rain", "Rivers", "Safari", "Scary Woods", "Sea", "Storm", "Swell", "Thunder", "Volcano", "Water", "Waterfall", "Waves", "Wind", "Wood", "Splash", "Ocean", "Lightning",
    "Bleep", "Buzzer", "Camera", "Click", "Clock", "Keyboard", "Drone", "Electricity", "Error", "High Tech", "Interface", "Laptop", "Light", "Movie Projector", "Radio", "Rewind", "Robot", "Static", "Tape Machine", "Tones", "TV", "Type", "Typewriter", "Ding",
    "Fart", "Laugh", "Cartoon", "Happy",
    "Alarm", "Alerts", "Beep", "Phone", "Phone Ring",
    "Bird", "Cat", "Chicken", "Cow", "Crickets", "Dog", "Farm", "Horse", "Insect", "Lion", "Monkey", "Mouse", "Roar", "Wild", "Wolf", "Pig",
    "Airport", "Bike", "Boat", "Bus", "Car", "Engine", "Flying", "Helicopter", "Motorcycle", "Race Car", "Ship", "Siren", "Skateboard", "Tanks", "Time Machine", "Tractor", "Traffic", "Train", "Tram", "Car Horn", "Truck",
    "Applause", "Baby", "Breathe", "Children", "Clapping", "Cough", "Crowd", "Cry", "Eat", "Falling", "Footsteps", "Gasp", "Heartbeat", "Hum", "Hurt", "Kiss", "Moan", "Party", "Scream", "Public Places", "Voices", "Yawn", "Teeth", "Thank You", "Sad", "Sneeze", "Terror", "Jump", "Cheer", "Walk", "Run",
    "Battle", "Battle Ambience", "Bomb", "Explosion", "Fight", "Gun", "Hit", "Medieval Battle", "Rocket", "Sword", "War", "Boom", "Crash",
    "Bass", "Bell", "Brass", "Chimes", "Choir", "Church Bell", "Cymbal", "DJ Record Scratch", "Drum", "Flute", "Guitar", "Horn", "Music", "Orchestra", "Rock", "Sleigh Bells", "Synth", "Violins", "Whistle", "Suspense Music", "Tap", "Piano",
    "Appliances", "Bar", "Bathroom", "Bedroom", "Church", "City", "Cleaning", "Cooking", "Creaking Floorboard", "Doorbell", "Doors", "Drink", "Emergency", "Food", "Garden", "Hall", "Hospital", "Kitchen", "Office", "Restaurant", "School", "Supermarket", "Sweeping", "Girl",
    "Arcade", "Arcade Sport", "Asteroid", "Athletics", "Ball", "Basketball", "Boxing", "Casino", "Game Over", "Game Show", "Goal", "Golf", "Light Saber", "Lose", "Monster", "Player", "Punch", "Slot Machine", "Soccer", "Space Shooter", "Sports", "Toy", "Video Game", "Win", "Correct", "Laser", "Wrong", "Shot",
    "Scary", "Balloon", "Break", "Bubbles", "Coin", "Construction", "Countdown", "Cute", "Fairy", "Festive", "Fireworks", "Intro", "Key", "Halloween", "Lock", "Magic", "Metal", "Movie", "Page", "Paper", "Police", "Pop", "Sci-Fi", "Sparkle", "Spell", "Squeak", "Tools", "White Noise", "Write", "Horror", "Money", "Glass", "Wow", "Drop"
]
output_dir = "mixkit_sounds_metadata"
os.makedirs(output_dir, exist_ok=True)

sound_data = []
sound_id = 4203  # Starting ID

for sub in subcategories:
    sub_url = f"{base_url}/free-sound-effects/{sub.replace(' ', '-').lower()}/"
    
    try:
        response = requests.get(sub_url)
        response.raise_for_status()
    except Exception as e:
        print(f"Failed to access {sub_url}: {e}")
        continue

    soup = BeautifulSoup(response.text, "html.parser")
    sound_cards = soup.select('div.item-grid__item')

    for card in tqdm(sound_cards, desc=f"Collecting from {sub}"):
        title_tag = card.select_one("h2.item-grid-card__title")
        audio_div = card.select_one('[data-audio-player-preview-url-value]')
        duration_tag = card.select_one('[data-test-id="duration"]')

        if not title_tag or not audio_div:
            continue

        title = title_tag.text.strip().replace(" ", "_").replace("/", "-")
        mp3_url = audio_div['data-audio-player-preview-url-value']

        # Parse duration
        duration_text = duration_tag.text.strip() if duration_tag else "0:00"
        try:
            mins, secs = map(int, duration_text.split(":"))
            duration_seconds = mins * 60 + secs
        except:
            duration_seconds = 0

        # Store metadata (no file downloading)
        sound_data.append({
            "id": sound_id,
            "name": title,
            "duration": duration_seconds,
            "url": mp3_url
        })
        sound_id += 1

# Save metadata to JSON
output_file = os.path.join(output_dir, "sounds_metadata.json")
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(sound_data, f, indent=2)

print(f"Metadata collection complete. Total sounds: {len(sound_data)}")
print(f"Saved to: {output_file}")


Metadata collection complete. Total sounds: 5129
Saved to: mixkit_sounds_metadata\sounds_metadata.json


In [42]:

import json
output_dir='mixkit_sounds_metadata'
os.makedirs(output_dir,exist_ok=True)
# Correct relative paths from your current working directory
with open('mixkit_sounds_metadata/sounds_metadata_filter.json', 'r', encoding='utf-8') as f:
    filter_data = json.load(f)
    print(f"filter_data {len(filter_data)}")

with open('mixkit_sounds_metadata/sounds_metadata.json', 'r', encoding='utf-8') as f:
    unfilter_data = json.load(f)
    
    print(f"unfilter_data {len(unfilter_data)}")

# # Create a lookup dict for fast access
unfilter_lookup = {item["name"]: item for item in unfilter_data}
 

# # Prepare result
result = []
index = 1
for f_item in filter_data:
    name = f_item["name"]
    if name in unfilter_lookup:
        u_item = unfilter_lookup[name]
        result.append({
            "id": index,
            "name": name,
            'duration':u_item['duration'],
            "description": f_item["description"],
            "url": u_item["url"]
        })
        index += 1

# Print the result
final_path=os.path.join(output_dir,'final_effects.json')
with open(final_path,'w',encoding='utf-8') as fi:
     json.dump(result, fi,indent=2)



filter_data 496
unfilter_data 5129
